In [53]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F

from sklearn.model_selection import train_test_split, KFold

import matplotlib.pyplot as plt

import plotly.graph_objects as go
import plotly.express as px

In [54]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [55]:
if torch.backends.mps.is_available():
    device = torch.device('mps')  # Mac Metal Performance Shaders
    print(f"Training on device: {device} (Mac GPU)")
elif torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"Training on device: {device} (NVIDIA GPU)")
else:
    device = torch.device('cpu')
    print(f"Training on device: {device} (CPU)")

print(f"PyTorch version: {torch.__version__}")
if device.type == 'mps':
    print("Using Mac Metal Performance Shaders for GPU acceleration")

print(f"PyTorch version: {torch.__version__}")
if device.type == 'mps':
    print("Using Mac Metal Performance Shaders for GPU acceleration")

Training on device: mps (Mac GPU)
PyTorch version: 2.9.0.dev20250715
Using Mac Metal Performance Shaders for GPU acceleration
PyTorch version: 2.9.0.dev20250715
Using Mac Metal Performance Shaders for GPU acceleration


In [56]:

class AttentionBlock(nn.Module):
    """Transformer layer for metabolic modeling"""
    def __init__(self,vocab_size=115,dim=6,num_heads=2):
        super(AttentionBlock, self).__init__()

        assert dim%num_heads==0, "model dimension must be divisible by number of heads"
        self.vocab_size = vocab_size
        self.d_model = dim
        
        self.layer_norm = nn.LayerNorm(dim)
        self.num_heads = num_heads
        self.k = dim//num_heads

        self.W_k = nn.Linear(dim,self.k,bias=False)
        self.W_q = nn.Linear(dim,self.k,bias=False)
        self.W_v = nn.Linear(dim,self.k,bias=False)
        self.W_o = nn.Linear(self.k,dim,bias=False)
        #self.W_c = nn.Linear(vocab_size,vocab_size,bias=False)

    def scaled_dot_product_attention(self,keys,queries,values):
        # Find the product if K and Q transpose and divide by the square root of the model dimension (d_model)

        pre_softmax_attention_matrix = torch.einsum('bij,bkj->bik', keys,queries)/np.sqrt(self.d_model)
        attention_matrix = torch.softmax(pre_softmax_attention_matrix,dim=-1)
        attention_output = torch.einsum( 'bij,bjk->bik' , attention_matrix, values)

        return attention_output, attention_matrix

    def forward(self,x,c):
        norm_x = self.layer_norm(x)
        #modified_c = self.W_c(c.transpose(-2,-1)).transpose(-2,-1)
        modified_c = c

        Q = self.W_k(norm_x)
        K = self.W_q(norm_x)
        V = self.W_v(norm_x)

        attention_output, attention_matrix = self.scaled_dot_product_attention(Q,K,V)

        #print(attention_matrix.size(),modified_c.size())

        attended_c = torch.einsum('bij,bjk->bik',attention_matrix,modified_c)
        
        #print(c.size(),attended_c.size())

        output_x = self.W_o(attention_output) + x*(1/self.num_heads)
        output_c = (attended_c + c)*(1/self.num_heads)

        return output_x, output_c


In [57]:

class MultiHeadAttentionBlock(nn.Module):
    """Multi-Head Attention layer for metabolic modeling"""
    def __init__(self,vocab_size=115,dim=6,num_heads=2):
        super(MultiHeadAttentionBlock, self).__init__()

        self.attention_blocks = nn.ModuleList([AttentionBlock(vocab_size,dim,num_heads) for _ in range(num_heads)])

    def forward(self,x,c):

        output_x = torch.zeros_like(x)
        output_c = torch.zeros_like(c)

        for attention_block in self.attention_blocks:
            o_x, o_c = attention_block(x,c)
            output_x += o_x
            output_c += o_c

        return output_x, output_c

        

In [58]:
class FeedForwardBlock(nn.Module):

    def __init__(self,d_model,inner_dim_multiplier,dropout=0.1):
        super(FeedForwardBlock, self).__init__()

        self.d_model = d_model+1
        self.inner_dim = inner_dim_multiplier*(d_model+1)

        self.layer_norm = nn.LayerNorm(self.d_model)

        self.linear_layer_1 = nn.Linear(self.d_model,self.inner_dim)
        self.linear_layer_2 = nn.Linear(self.inner_dim,self.d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self,x,c):

        y = torch.cat((x,c),2)
        
        norm_y = self.layer_norm(y)
        
        norm_y = self.linear_layer_1(norm_y)
        norm_y = F.relu(norm_y)
        norm_y = self.linear_layer_2(norm_y)

        return norm_y + y

In [59]:

class TransformerBlock(nn.Module):
    """Embedding layer + Attention Block + FeedForward Layer"""
    def __init__(self,vocab_size=115,dim=6,num_heads=2,inner_dim_multiplier=5):
        super(TransformerBlock, self).__init__()

        self.d_model = dim
        self.vocab_size = vocab_size

        self.inp_embedding = nn.Embedding(vocab_size,dim)

        self.attention_block = MultiHeadAttentionBlock(vocab_size,dim,num_heads)

        self.feedforward_block = FeedForwardBlock(dim,inner_dim_multiplier)

        self.linear_layer_1 = nn.Linear(vocab_size,vocab_size)

        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.32)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.32)
    
    
    def forward(self,c):

        batch_size, vocab_size, _ = c.size()

        # y = torch.randint(0, vocab_size, (batch_size, vocab_size))
        # for k in range(vocab_size):
        #     y[:,k] = k

        y = torch.arange(vocab_size,device=device).unsqueeze(0).expand(batch_size, -1)
        
        x = self.inp_embedding(y)
        # print(x.size())
        
        output_x, output_c = self.attention_block(x,c)

        output_y = self.feedforward_block(output_x,output_c)

        return output_y[:,:,-1].unsqueeze(-1)

        #return output_c


In [60]:
class TransformersSeries(nn.Module):
    def __init__(self,vocab_size=115,dim=6,num_heads=2,inner_dim_multiplier=5,num_transformers=2):
        super(TransformersSeries, self).__init__()

        self.d_model = dim
        self.vocab_size = vocab_size
        self.num_transformers = num_transformers

        self.inp_embedding = nn.Embedding(vocab_size,dim)

        self.attention_blocks = nn.ModuleList([MultiHeadAttentionBlock(vocab_size,dim,num_heads) for _ in range(num_transformers)])

        self.feedforward_blocks = nn.ModuleList([FeedForwardBlock(dim,inner_dim_multiplier) for _ in range(num_transformers)])

        self.linear_layer_1 = nn.Linear(vocab_size,vocab_size)

        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    
    def forward(self,c):

        batch_size, vocab_size, _ = c.size()

        y = torch.arange(vocab_size,device=device).unsqueeze(0).expand(batch_size, -1)
        
        x = self.inp_embedding(y)

        
        # The series of transformer layers
        for i in range(self.num_transformers):

            output_x, output_c = self.attention_blocks[i](x,c)

            output_y = self.feedforward_blocks[i](output_x,output_c)

            x = output_y[:,:,:-1]
            c = output_y[:,:,-1].unsqueeze(-1)

        return output_y[:,:,-1].unsqueeze(-1)



In [61]:
inps_test_ = torch.load('inputs_test.pt')
outs_test_ = torch.load('outputs_test.pt')


In [62]:
input_cols = [
        'EX_glc__D_e', 'EX_fru_e', 'EX_lac__D_e', 'EX_pyr_e', 'EX_ac_e',
        'EX_akg_e', 'EX_succ_e', 'EX_fum_e', 'EX_mal__L_e', 'EX_etoh_e',
        'EX_acald_e', 'EX_for_e', 'EX_gln__L_e', 'EX_glu__L_e',
        'EX_co2_e', 'EX_h_e', 'EX_h2o_e', 'EX_nh4_e', 'EX_o2_e', 'EX_pi_e'
    ]

output_cols = [
        'ACALD_flux',
        'ACALDt_flux',
        'ACKr_flux',
        'ACONTa_flux',
        'ACONTb_flux',
        'ACt2r_flux',
        'ADK1_flux',
        'AKGDH_flux',
        'AKGt2r_flux',
        'ALCD2x_flux',
        'ATPM_flux',
        'ATPS4r_flux',
        'Biomass_Ecoli_core_flux',
        'CO2t_flux',
        'CS_flux',
        'CYTBD_flux',
        'D_LACt2_flux',
        'ENO_flux',
        'ETOHt2r_flux',
        'EX_ac_e_flux',
        'EX_acald_e_flux',
        'EX_akg_e_flux',
        'EX_co2_e_flux',
        'EX_etoh_e_flux',
        'EX_for_e_flux',
        'EX_fru_e_flux',
        'EX_fum_e_flux',
        'EX_glc__D_e_flux',
        'EX_gln__L_e_flux',
        'EX_glu__L_e_flux',
        'EX_h_e_flux',
        'EX_h2o_e_flux',
        'EX_lac__D_e_flux',
        'EX_mal__L_e_flux',
        'EX_nh4_e_flux',
        'EX_o2_e_flux',
        'EX_pi_e_flux',
        'EX_pyr_e_flux',
        'EX_succ_e_flux',
        'FBA_flux',
        'FBP_flux',
        'FORt2_flux',
        'FORti_flux',
        'FRD7_flux',
        'FRUpts2_flux',
        'FUM_flux',
        'FUMt2_2_flux',
        'G6PDH2r_flux',
        'GAPD_flux',
        'GLCpts_flux',
        'GLNS_flux',
        'GLNabc_flux',
        'GLUDy_flux',
        'GLUN_flux',
        'GLUSy_flux',
        'GLUt2r_flux',
        'GND_flux',
        'H2Ot_flux',
        'ICDHyr_flux',
        'ICL_flux',
        'LDH_D_flux',
        'MALS_flux',
        'MALt2_2_flux',
        'MDH_flux',
        'ME1_flux',
        'ME2_flux',
        'NADH16_flux',
        'NADTRHD_flux',
        'NH4t_flux',
        'O2t_flux',
        'PDH_flux',
        'PFK_flux',
        'PFL_flux',
        'PGI_flux',
        'PGK_flux',
        'PGL_flux',
        'PGM_flux',
        'PIt2r_flux',
        'PPC_flux',
        'PPCK_flux',
        'PPS_flux',
        'PTAr_flux',
        'PYK_flux',
        'PYRt2_flux',
        'RPE_flux',
        'RPI_flux',
        'SUCCt2_2_flux',
        'SUCCt3_flux',
        'SUCDi_flux',
        'SUCOAS_flux',
        'TALA_flux',
        'THD2_flux',
        'TKT1_flux',
        'TKT2_flux',
        'TPI_flux'
    ]

all_data = input_cols + output_cols

In [10]:
trained_model = TransformersSeries()  # Must recreate model structure
trained_model.load_state_dict(torch.load('trained_model.pth'))
trained_model.to(device)

TransformersSeries(
  (inp_embedding): Embedding(115, 6)
  (attention_blocks): ModuleList(
    (0-1): 2 x MultiHeadAttentionBlock(
      (attention_blocks): ModuleList(
        (0-1): 2 x AttentionBlock(
          (layer_norm): LayerNorm((6,), eps=1e-05, elementwise_affine=True)
          (W_k): Linear(in_features=6, out_features=3, bias=False)
          (W_q): Linear(in_features=6, out_features=3, bias=False)
          (W_v): Linear(in_features=6, out_features=3, bias=False)
          (W_o): Linear(in_features=3, out_features=6, bias=False)
        )
      )
    )
  )
  (feedforward_blocks): ModuleList(
    (0-1): 2 x FeedForwardBlock(
      (layer_norm): LayerNorm((7,), eps=1e-05, elementwise_affine=True)
      (linear_layer_1): Linear(in_features=7, out_features=35, bias=True)
      (linear_layer_2): Linear(in_features=35, out_features=7, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (linear_layer_1): Linear(in_features=115, out_features=115, bias=True)
)

In [11]:
trained_model_12_3_5_2 = TransformersSeries(115,12,3,5,2)
trained_model_12_3_5_2.load_state_dict(torch.load('trained_model_12_3_5_2.pth'))

trained_model_12_3_5_2.to(device) 


TransformersSeries(
  (inp_embedding): Embedding(115, 12)
  (attention_blocks): ModuleList(
    (0-1): 2 x MultiHeadAttentionBlock(
      (attention_blocks): ModuleList(
        (0-2): 3 x AttentionBlock(
          (layer_norm): LayerNorm((12,), eps=1e-05, elementwise_affine=True)
          (W_k): Linear(in_features=12, out_features=4, bias=False)
          (W_q): Linear(in_features=12, out_features=4, bias=False)
          (W_v): Linear(in_features=12, out_features=4, bias=False)
          (W_o): Linear(in_features=4, out_features=12, bias=False)
        )
      )
    )
  )
  (feedforward_blocks): ModuleList(
    (0-1): 2 x FeedForwardBlock(
      (layer_norm): LayerNorm((13,), eps=1e-05, elementwise_affine=True)
      (linear_layer_1): Linear(in_features=13, out_features=65, bias=True)
      (linear_layer_2): Linear(in_features=65, out_features=13, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (linear_layer_1): Linear(in_features=115, out_features=115, bias=Tr

In [12]:
models = [trained_model,trained_model_12_3_5_2]
model_configs = ['dmodel=6,num_heads=2,inner_dim_multiplier=5,num_transformers=2','dmodel=12,num_heads=3,inner_dim_multiplier=5,num_transformers=2']

In [13]:
def plot_data(simulation,model):
    plt.figure(figsize=(6*2, 4*2))
    j = simulation
    k = model
    pred_ops = models[k](inps_test_[j,:,:].unsqueeze(0))
    target_ops = outs_test_[j,:,:].unsqueeze(0)
    inputs = inps_test_[j,:,:].unsqueeze(0)
    plt.stem(inputs[0,:,0].cpu().detach().numpy(),'g',label='Input Concentrations')
    plt.plot(pred_ops[0,:,0].cpu().detach().numpy(),color='red',label='Predicted Concentrations')
    plt.plot(target_ops[0,:,0].cpu().detach().numpy(),color='black',label='Actual Concentrations')
    plt.ylim([-50,100])
    plt.legend()
    plt.title(f"Simulation {j}:{model_configs[k]}")
    plt.grid()

In [51]:
interactive_plot = interactive(plot_data, simulation=(0, inps_test_.shape[0] - 1),model=(0,1))
output = interactive_plot.children[-1]
interactive_plot

interactive(children=(IntSlider(value=14932, description='simulation', max=29864), IntSlider(value=0, descript…

In [74]:


xx = np.array([i for i in range(20)])


# Create plotly figure
fig_plotly = go.Figure()
inputs = inps_test_[0,:20,:].unsqueeze(0)
inp = inputs[0,:,0].cpu().detach().numpy()

pred_ops = models[0](inps_test_[0,:,:].unsqueeze(0))
target_ops = outs_test_[0,:,:].unsqueeze(0)

fig_plotly.add_trace(go.Scatter(
    x=xx,
    y=inp,
    mode='markers',
    marker=dict(
        size=12,
        color='green',
    ),
    text=[f'{all_data[i]}' 
            for i in xx],
    hovertemplate='%{text}<extra></extra>',
    name='Input Concentrations'
))

xxx = np.array([i for i in range(115)])

fig_plotly.add_trace(go.Scatter(
    x=xxx, 
    y = pred_ops[0,:,0].cpu().detach().numpy(), 
    mode='lines+markers', 
    text = [f'{all_data[i]}' 
            for i in xxx],
    hovertemplate='%{text}<extra></extra>',
    name='Predicted Concentrations'
    ))

fig_plotly.add_trace(go.Scatter(
    x=xxx, 
    y = target_ops[0,:,0].cpu().detach().numpy(), 
    mode='lines+markers', 
    text = [f'{all_data[i]}' 
            for i in xxx],
    hovertemplate='%{text}<extra></extra>',
    name='Predicted Concentrations'
    ))

fig_plotly.show()


In [45]:
xxx = np.linspace(0,114,115)
xxx

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 112., 113., 114.])

In [15]:
pred_outs_ = models[0](inps_test_)
pred_outs_.size()

torch.Size([29865, 115, 1])

In [16]:
outs_test_.size()

torch.Size([29865, 115, 1])

In [17]:
def plot_pred_actual(metabolite,simulation,model):  
    pred_outs_ = models[model](inps_test_)
    plt.scatter(pred_outs_[:,metabolite,:].cpu().detach().numpy(),outs_test_[:,metabolite,:].cpu().detach().numpy(),color='r',alpha=0.1)
    plt.scatter(pred_outs_[simulation,metabolite,:].cpu().detach().numpy(),outs_test_[simulation,metabolite,:].cpu().detach().numpy(),color='k',alpha=0.9)
    plt.xlim([-50,100])
    plt.ylim([-50,100])
    plt.grid()
    plt.xlabel('Predicted Output')
    plt.ylabel('Actual Output')
    plt.title('Input Output Characterstics')
    plt.show()

In [18]:
interactive_plot = interactive(plot_pred_actual, metabolite=(0, inps_test_.shape[1] - 1), simulation = (0, inps_test_.shape[0] - 1), model = (0,1))
output = interactive_plot.children[-1]
interactive_plot

interactive(children=(IntSlider(value=57, description='metabolite', max=114), IntSlider(value=14932, descripti…